# Data preprocessing: more art than science?
## Written by Nadia Blostein and Cesare Spinoso

## Contents of this notebook:
<ol>
<li>Load and examine your data</li>
<li>Merging two dataframes</li>
<li>Removing features that you do not need</li>
<li>Making your data machine-readable</li>
<li>Handling not available (NA) and inf data</li>
<li>Data visualization</li>
<li>2D image visualization</li>
</ol>

# Setup
Fetch the dataset that you'll be working with throughout this assignment.


In [ ]:
!git clone https://github.com/NadiaBlostein/Open-Access-HCP-Data.git

Examine your directory structure with the `ls` command. To invoke this command from a jupyter notebook in Google Colab, the `ls` command should be preceded with `!`. 

In [ ]:
!ls

Change the working directory of the notebook to within the folder `Open-Access-HCP-Data`. The `cd` command should be preceded with `%`.


In [ ]:
%cd Open-Access-HCP-Data 
!ls

In this assignment, we will be working with data from the Human Connectome Project (HCP). You can read more about the data [here](https://github.com/NadiaBlostein/McMedHacks2022_Prep_Week_3_Assignment#readme). Specifically, you will preprocess `.csv` (in the `HCP_csv_data` folder) and `.png` files (in the `HCP_2D_slices_MRI_data` folder).

In [ ]:
!ls HCP_2D_slices_MRI_data

In [ ]:
!ls HCP_csv_data 

# 1. Load and examine your data

### Preliminary examination of your data

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("HCP_csv_data/unrestricted_HCP_behavioral.csv")

**Question 1:** How many subjects (rows) and features (columns) do you have? PS: you cannot always assume that your subjects are rows and features are columns, and sometimes there may be rows with useless information that you need to remove.

In [ ]:
#@title
print(f"Num subjects: {df.shape[0]}")
print(f"Num features: {df.shape[1]}")

Feel free to examine your data frame in the empty code cell below. Examples of valuable commands for a preliminary look at one's data frame: `df.info()`, `df.describe()`, `df.columns()`, `df.head()`, `df.tail`.

Next, we will select the `Gender` column to count how many instances we have of each value.

In [ ]:
print(f"unique values of column: {df['Gender'].unique()}") # prints list of unique values in a feature / column
print(f"\nvalue counts of column:\n{df['Gender'].value_counts()}") # counts how many times each unique value in a feature / column occurs

**Question 2:** Is there another way to count how many males and females are in this dataset? 

In [ ]:
#@title
print(f"Number females: {len(df[df['Gender']=='F'])}")
print(f"Number males: {len(df[df['Gender']=='M'])}")

**Question 3:** Display only the rows associated with female subjects.

In [ ]:
#@title
df[df['Gender']=="F"]

### A side note on documentation
Woah! So many columns and abbreviations! What do they all mean? Make sure you know where your [dataset's documentation](https://wiki.humanconnectome.org/display/PublicData/HCP-YA+Data+Dictionary-+Updated+for+the+1200+Subject+Release#HCPYADataDictionaryUpdatedforthe1200SubjectRelease-Instrument:Demographics) is.

Unfortunately, thorough documentation is not always available. Some data types are also very field-specific and require the help of experts, which is part of what makes machine learning so wonderfully interdisciplinary.

Let's take a look at what features we have here:

In [ ]:
print(df.columns)

**Question 4:** The script above is not printing ALL of the columns... How do we fix that to be able to see all the features in the dataset?

In [ ]:
#@title
for col in df.columns: print(col)

### Selecting the features you want to work with:
As you saw above, this dataframe has 499 features. Often, you will only want to work with a subset of the features of a dataframe, so you will have to create a new dataframe with this subset.

In [ ]:
basics = ['Subject','Gender','Age','PSQI_BedTime']
df[basics]

Let's also add all of some cognitive variables to the mix! Specifically, we'll select the measures related to fluid intelligence (they start with `PMAT` for Penn Matrix Test) and impulsivity (they start with `DDisc` for Delay Discounting)

In [ ]:
cognition = ['Subject','Gender','Age','PSQI_BedTime']
for col in df.columns:
    if (col.find("PMAT")!=-1 or col.find("DDisc")!=-1):
        cognition.append(col)
print(f"List of variables we will be looking at: {cognition}") # PS: f-strings will be very useful for you in your Python journey!

**Question 5:** Now that we made a list of all of the features we want to examine, select this subset of our data and make it a separate dataframe called `df_cognition`.

In [ ]:
#@title
df_cognition = df[cognition]
df_cognition.head()

In [ ]:
df_cognition.shape

# 2. Merging two dataframes

For our dataset of 1206 subjects, we have information about their gender, age range and a variety of cognitive measures. It would be interesting to integrate some other data as well. You have been provided with a separate file that contains brain structure volume data obtained from the neuroimaging data of these same subjects (note that not all subjects in the HCP have neuroimaging data) (see [README](https://github.com/NadiaBlostein/McMedHacks2022_Prep_Week_3_Assignment#readme) to learn more about how volumes were obtained).

In [ ]:
df_volumes = pd.read_csv("HCP_csv_data/HCP_volumes.csv")

**Question 6:** How many subjects and features does `df_volumes` have?

In [ ]:
#@title
print(f"Num subjects: {df_volumes.shape[0]}")
print(f"Num features: {df_volumes.shape[1]}")

**Question 7:** Print the mean and standard deviation of total brain volume (TBV) of this sample. Note that the unit is in mm3.

In [ ]:
#@title
print(f"Mean TBV: {df_volumes['TBV'].mean()}")
print(f"TBV standard deviation: {df_volumes['TBV'].std()}")

**Question 8:** List all the subjects whose TBV is above average.

In [ ]:
#@title
df_volumes[df_volumes['TBV']>df_volumes['TBV'].mean()]

Ok! Let's merge our dataframes. One problem is that our behavioral data has 1206 subjects and our volume data has 1086 subjects. 

**Question 9:** Create a new dataframe called `df_final` where we only keep the subjects for which we have all of our features.

In [ ]:
#@title
df_final=pd.merge(df_cognition,df_volumes, on='Subject')
df_final.head()

In [ ]:
print(df_final.shape)

Mission accomplished! We have 1086 and 35 features (22 features + 12 features - `Subject` features which is shared by both dfs).

# 3. Removing features that you do not need

**Question 10:** Find and remove any duplicate columns.

In [ ]:
#@title
cols_to_drop=[]
for i in range(df_final.shape[1]):
    for j in range(i+1,df_final.shape[1]):
        col1=df_final.columns[i]
        col2=df_final.columns[j]
        if (df_final[col1].equals(df_final[col2])):
            print(f"Duplicate columns: {col1, col2}")
            cols_to_drop.append(col2)
df_final.drop(cols_to_drop, inplace=True, axis=1)

In [ ]:
print(df_final.shape) 

# 4. Making your data machine-readable

To be machine-readable, your variables need to be numerical. Want to check?

In [ ]:
df_final.dtypes

We have 3 columns that are non-numerical: `Gender`,`Age`,`PSQI_BedTime`. Let's figure out how to handle them, one at a time.

### One-hot encoding or binarizing your data
First, we know that the `Gender` column is categorical and has two unique values: `M` or `F`. 

**Question 11:** Replace all of your `M` values with 1 and `F` values with 2. Hint: `.replace()` can be quite helpful.

In [ ]:
#@title
df_final['Gender'] = df_final['Gender'].replace('M',1)
df_final['Gender'] = df_final['Gender'].replace('F',2)

**Quick note on one-hot encoding:** Suppose that you actually had more than 2 numerical values for this feature (e.g. `M`,`F`,`other`). If you just convert categorical variables to numerical values (ex: `M`=1,`F`=2,`other`=3), you give a "distance" to the relationship between variables. For instance, since 1 is closer to 2 than to 3, you are telling your machine that `M` is "closer" to `F` (`distance = 2 - 1 = 1`) than to `other` (`distance = 3 - 1 = 2`). One-hot encoding is a way to make sure the categories remain independant and we strongly suggest that you read more about it: "[A representation of categorical variables as binary vectors](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/)"

### Parsing strings in your data

#### Handling the `Age` feature

Remember that our `Age` feature is also non-numerical!

In [ ]:
df_final['Age'].value_counts()

For the sake of this exercise, let's replace 36+ with a range of 36-40.

In [ ]:
df_final['Age'] = df_final['Age'].replace("36+",'36-40')
df_final['Age'].value_counts()

Our age range values are organized very clearly: `minimum age–maximum age`. These age range strings can therefore be split around the `-` such that you create two new columns: one for `minimum age` and one for `maximum age`.

In [ ]:
fix_age = df_final['Age'].str.split('-', 1, expand=True)
fix_age.columns = ['min','max']
fix_age["min"] = fix_age['min'].astype(float)
fix_age["max"] = fix_age['max'].astype(float)
fix_age

**Question 12:** Using the code in the cell above, replace the age range string for each subject with the mean of the subject's respective age range.

In [ ]:
#@title
fix_age['mean'] = (fix_age['max']+fix_age['min'])/2
df_final['Age'] = fix_age['mean']

In [ ]:
df_final.head(3)

#### Handling the `PSQI_BedTime` feature

Convert your bed time variable from HH:MM:SS to seconds! You can walk through the code to figure out what each step does.
`

In [ ]:
ftr = [3600,60,1]
for i in range(len(df_final['PSQI_BedTime'])):
    x = sum([a*b for a,b in zip(ftr, map(int,df_final['PSQI_BedTime'][i].split(':')))])
    df_final['PSQI_BedTime'][i] = x

#### A note on other strings that often crop up in dataframes and need to be replaced with numbers!
`df_final = df_final.replace('FALSE',0)` \
`df_final = df_final.replace('TRUE',1)` \
`df_final = df_final.replace(False,0)` \
`df_final = df_final.replace(True,1)` \
`df_final = df_final.replace('0',0)` # example of random spaces \
`df_final = df_final.replace(' ',np.NaN)` # example of random spaces

**Question 13:** Write a code that makes sure that every column is of type float!

In [ ]:
#@title
for col in df_final.columns:
    df_final[col] = df_final[col].astype(float)

# 5. Handling not available (NA) and inf data:

Sometimes, Python will convert some of your values to + or - infinity, which will result in downstream errors. Convert them to NA, and then handle them as NA values.

In [ ]:
df_final = df_final.replace([np.inf, -np.inf], np.nan)

Next, you need to deal with your NA values. 

**Question 14:** How many nas do you have?

In [ ]:
#@title
df_final.isna().sum()

There is a variety of ways to handle NA data. The most simple approach is to replace NA data with the median (or mean) value of the feature of interest. There are [other](https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779) [more](https://arxiv.org/abs/1804.11087) sophisticated data imputation techniques out there, many of which actually leverage machine learning tools (so meta)!

**Question 15:** Replace the NA values of this dataframe with the feature-specific median (the median is more robust against outliers than the mean is).

In [ ]:
#@title
for col in df_final.columns:
    df_final[col].fillna(df_final[col].median(), inplace=True)

In [ ]:
df_final.isna().sum().sum()
# note the difference between df_final.isna().sum() and df_final.isna().sum().sum()

# 6. Data visualization

Data visualization is a wonderful way to get to know your data in order to plan a relevant analysis or find an appropriate machine learning application. [Matplotlib](https://matplotlib.org/) and [seaborn](https://seaborn.pydata.org/) are two canonical data visualization tools that you can use in Python.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#### Scatter plots

In [ ]:
plt.rcParams["figure.figsize"] = (10,5) # adjust figure size
plt.scatter(df_final["PSQI_BedTime"],df_final["TBV"]) # plt.scatter(x, y)
plt.ylabel("TBV (mm3)") # name y-axis
plt.xlabel("Bedtime (seconds)") # name x-axis
plt.title("Total brain volume (TBV) as a function of bed time") # figure title

**Question 16:** Make a scatter plot of total brain volume (TBV) as a function of bed time, by gender.

In [ ]:
#@title
plt.rcParams["figure.figsize"] = (10,5) # adjust figure size
plt.scatter(df_final["PSQI_BedTime"][df_final["Gender"]==2],df_final["TBV"][df_final["Gender"]==2],color='turquoise')
    # plot female datapoints first, in turquoise
plt.scatter(df_final["PSQI_BedTime"][df_final["Gender"]==1],df_final["TBV"][df_final["Gender"]==1],color='coral')
    # plot female datapoints first, in coral
plt.ylabel("TBV (mm3)") # name y-axis
plt.xlabel("Bedtime (seconds)") # name x-axis
plt.title("Total brain volume (TBV) as a function of bed time, by gender") # figure title
plt.legend(['Females','Males']) # legend

#### Histogram plot

In [ ]:
sns.displot(df_final,x='TBV',kind='kde',fill=True) # plot your histogram
plt.ylabel("Density") # name y-axis
plt.xlabel("TBV (mm3)") # name x-axis
plt.title("Distribution of total brain volume (TBV) in our sample") # figure title
plt.gcf().set_size_inches(8, 5) # another way to adjust figure size

In [ ]:
df_final.columns

In [ ]:
sns.histplot(df_final,x='Str_Left',fill=True,color='orange')
    # first, plot the distribution of the left striatum in orange
sns.histplot(df_final,x='Thal_Left',fill=True,color='turquoise')
    # second, plot the distribution of the left thalamus volume in turquoise
plt.legend(['Left striatum','Left thalamus'])
plt.ylabel("Density")
plt.xlabel("Structure-specific volume (mm3)")
plt.title("Distribution of different structure volumes in our sample")
plt.gcf().set_size_inches(8, 5) # adjust figure size
plt.show()

**Question 17:** Generate two smoothed and superimposed histograms of bed times in subjects that are above and below 30 years.

In [ ]:
#@title
df_final["Over 25 years"]=df_final["Age"]>25
sns.displot(df_final,x='TBV',hue='Over 25 years',kind='kde',fill=True)
plt.ylabel("Density")
plt.xlabel("TBV (mm3)")
plt.title("Distribution of total brain volume (TBV) in our sample for people who are below and above 25 years of age")
plt.gcf().set_size_inches(14, 6) 

Notice that brains seem to shrink with age in our dataset... Let's look at the numbers:

In [ ]:
TBV_below_25=df_final['TBV'][df_final["Age"]<=25].mean()
TBV_above_25=df_final['TBV'][df_final["Age"]>25].mean()
print(f"Average TBV for people below or equal to 25 years of age: {round(TBV_below_25,0)} mm3") 
print(f"Average TBV for people above 25 years of age: {round(TBV_above_25,0)} mm3")

Interesting! However, does our sample actually have a similar amout of people who are below and above the age of 25? 

**Question 18:** Count how many people are <= 25 years, and how many people are > 25 yrs.

In [ ]:
subj_above_25=df_final[df_final["Age"]>25].shape[0]
subj_below_25=df_final[df_final["Age"]<=25].shape[0]
print(f"We have {subj_below_25} subjects below or at 25 years of age and {subj_above_25} subjects above 25 years of age.")

# 7. 2D image visualization
Let's visualize the `.png` files that come with this repository. We can do this by opening the image with the `PIL.Image` module seen in the previous lectures/assigments.

In [ ]:
# Get the image as an array
from PIL import Image
import os

# Try changing the 
png_file_path = os.path.join("HCP_2D_slices_MRI_data", "HCP_102109_T1w_acpc_dc_restore_brain_t1_axial.png")
png_array = np.array(Image.open(png_file_path))

Let's inspect the `png_array` object. The `Image.open` method opens the `.png` as a numpy array, so we can use all the `numpy` array methods on it. 

**Qusetion 19:** Create a function `plot_png_array` which takes as parameters a list of `png_array`'s, a `title` and the figure size `figsize`. Hint: Look at the previous week's assignment and plots each element of the array as a grayscale.

In [ ]:
# Function that plots a *list* of numpy arrays in grayscale
def plot_png_array(png_array_list, title, figsize=(10, 10)):
  for png_array in png_array_list:
    # Plot the image with imgshow
    fg, ax = plt.subplots(figsize=(10, 10))

    ax.imshow(png_array, cmap='gray')

    ax.set_title(title)

    plt.show()

Use the function you just created to plot the `png_array` assigned above.

In [ ]:
plot_png_array([png_array,], "Brain")

Inspect the following code. What do you think the `reduce_png_array_dim` does?

In [ ]:
def reduce_png_array_dim(png_array, kernel_height=2, kernel_width=2):
  cropped_png_array = []
  for channel_idx in range(0, png_array.shape[-1]):
    cropped_channel_png_array = []
    for i in range(0, png_array.shape[0], kernel_height):
      cropped_array = []
      for j in range(0, png_array.shape[1], kernel_width):
        cropped_array.append(png_array[i:i+kernel_height, j:j+kernel_width, channel_idx].mean())
      cropped_channel_png_array.append(cropped_array)
    cropped_channel_png_array = np.array(cropped_channel_png_array)
    cropped_png_array.append(cropped_channel_png_array)
  cropped_png_array = np.array(cropped_png_array)
  # Numpy shape fix + convert to integer
  return np.moveaxis(cropped_png_array, 0, -1).astype(np.int32)

In [ ]:
cropped_png_array = reduce_png_array_dim(png_array)

Hint: Look at the input before and after `reduce_png_array_dim`. What do you observe in terms of the 

In [ ]:
# Replot the original and cropped image
plot_png_array([png_array, cropped_png_array], "Brain")